In [ ]:
!pip install azure-ai-formrecognizer
!pip install google-colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.1/300.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.7 MB/s eta 0:00:00


In [ ]:
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential
import re

In [ ]:
def invoice_reader(pdf_path):
  endpoint = "https://idpcustomstest.cognitiveservices.azure.com/"
  key = "df758a658ccf464690f4071453f06374"
  form_recognizer_client = FormRecognizerClient(endpoint, AzureKeyCredential(key))

  # Get the PDF path from the uploaded files

  # Analyze the PDF document
  with open(pdf_path, "rb") as pdf_file:
      poller = form_recognizer_client.begin_recognize_invoices(pdf_file)
      result = poller.result()
  return result

In [ ]:
pdf_path = "/content/SICSY2309001500 - CI.pdf"
raw_form = invoice_reader(pdf_path)

In [ ]:
print(raw_form)

[RecognizedForm(form_type=prebuilt:invoice, fields={'CustomerAddress': FormField(value_type=string, label_data=None, value_data=FieldData(page_number=1, text=180, Serangoon Rd, Singapore 218057, bounding_box=[Point(x=0.3302, y=1.9289), Point(x=1.656, y=1.9289), Point(x=1.656, y=2.2861), Point(x=0.3302, y=2.2861)], field_elements=None), name=CustomerAddress, value='180, Serangoon Rd, Singapore 218057', confidence=0.945), 'CustomerAddressRecipient': FormField(value_type=string, label_data=None, value_data=FieldData(page_number=1, text=ASIAN ARTS CRAFTS, bounding_box=[Point(x=0.3268, y=1.7214), Point(x=1.7001, y=1.7214), Point(x=1.7001, y=1.8317), Point(x=0.3268, y=1.8317)], field_elements=None), name=CustomerAddressRecipient, value='ASIAN ARTS CRAFTS', confidence=0.946), 'CustomerName': FormField(value_type=string, label_data=None, value_data=FieldData(page_number=1, text=ASIAN ARTS CRAFTS, bounding_box=[Point(x=0.3268, y=1.7214), Point(x=1.7001, y=1.7214), Point(x=1.7001, y=1.8317), Poi

In [ ]:
import pandas as pd


def invoice_result_parser_for_top_fields(result):
  # Initialize an empty list to store the name and value pairs
  name_value_pairs = []
  dict_result = [page.to_dict() for page in result]

  # Iterate over the dictionary d
  for page_data in dict_result:
      fields = page_data['fields']
      for field_name, field_data in fields.items():
          name_value_pairs.append({'Name': field_name, 'Value': field_data['value']})

  # Create a DataFrame from the list of name and value pairs
  df = pd.DataFrame(name_value_pairs)
  df_json = json.loads(df.to_json(orient="records"))

  # Remove the "Items" row from the JSON data
  for i in range(len(df_json)):
      if df_json[i]["Name"] == "Items":
          del df_json[i]
          break

  # Convert the modified JSON data to a dictionary with "Name" as key and "Value" as value
  df_dict = {item["Name"]: item["Value"] for item in df_json}
  return df_dict

In [ ]:
import json
top_part_of_invoice = invoice_result_parser_for_top_fields(raw_form)
print(top_part_of_invoice)

{'CustomerAddress': '180, Serangoon Rd, Singapore 218057', 'CustomerAddressRecipient': 'ASIAN ARTS CRAFTS', 'CustomerName': 'ASIAN ARTS CRAFTS', 'InvoiceDate': None, 'InvoiceId': 'DT', 'ServiceEndDate': 1688774400000, 'VendorAddress': '1240, CHAH RAHAT, GALI GULIAN DELHI 110006 (INDIA)', 'VendorAddressRecipient': 'DIGAMBER EXPORTS', 'VendorName': 'DIGAMBER EXPORTS'}


In [ ]:
import pickle
import pandas as pd


def rename_product_code_to_hsn(df):
    """
    Renames the 'PRODUCT CODE' column to 'HSN CODE' if more than 30% of its values match the given HS codes.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the 'PRODUCT CODE' column.
    - hs_codes_df (pd.DataFrame): The DataFrame containing the HS codes to compare against.

    Returns:
    - pd.DataFrame: The modified DataFrame with the column name possibly changed.
    """

    with open("/content/HSCODE.pkl", "rb") as f:
        hs_codes = pickle.load(f)

    # Convert the pickle data to a DataFrame, assuming hs_codes is a list or similar iterable
    hs_codes_df = pd.DataFrame(hs_codes, columns=['HS Code'])


    # Check if 'PRODUCT CODE' is in the DataFrame
    if 'PRODUCT CODE' not in df.columns:
        print("The DataFrame does not contain a 'PRODUCT CODE' column.")
        return df

    # Convert the 'HS CODE' column to a list
    hs_codes = hs_codes_df['HS CODE'].tolist()

    # Calculate the total number of matches
    matches = df['PRODUCT CODE'].isin(hs_codes).sum()

    # Calculate the percentage of matches
    match_percentage = matches / len(df) * 100

    # If more than 30% of 'PRODUCT CODE' values are in hs_codes, rename the column
    if match_percentage > 30:
        df = df.rename(columns={'PRODUCT CODE': 'HSN CODE'})

    return df


In [ ]:
def extract_hsn_codes(description):
    # Define the regular expression pattern to match "HS CODE" followed by any separator
    pattern = r'HS\s*CODE\s*[:\-]?\s*'
    # Find all occurrences of the pattern in the description
    matches = re.findall(pattern, description)

    # Extract the HS codes after each match
    hsn_codes = []
    for match in matches:
        # Find all HS codes after the match (separated by commas or spaces)
        hs_codes = re.findall(r'\d{4,10}', description.split(match, 1)[1])
        hsn_codes.extend(hs_codes)

    return hsn_codes

In [ ]:
# Assuming 'd' is your list of dictionaries
def invoice_result_parser_for_items(result):
  data = []
  dict_result = [page.to_dict() for page in result]
  for page in dict_result:
      for item in page['fields']['Items']['value']:
          # Extract the values from the nested dictionaries
          row = {k: v['value'] for k, v in item['value'].items()}
          data.append(row)

  # Convert the list of dictionaries into a DataFrame
  df = pd.DataFrame(data)
  #df['HS CODE'] = df['Description'].apply(extract_hsn_codes)
  #if 'PRODUCT CODE' in df.columns:
  #  df = rename_product_code_to_hsn(df, hs_codes)
  #else:
  #  df['HS CODE'] = df['Description'].apply(extract_hsn_codes)
  json_object_invoice_tables = df.to_json(orient='records')
  return json_object_invoice_tables

In [ ]:
item_part_of_invoice = invoice_result_parser_for_items(raw_form)
print(item_part_of_invoice)

[{"Amount":11066.76,"Description":"BRASS ASSORTED STATUES","ProductCode":"83061000","UnitPrice":9.0},{"Amount":81.12,"Description":"BRASS ITEMS","ProductCode":"83061000","UnitPrice":3.38},{"Amount":27.0,"Description":"KADA IMITATION","ProductCode":"1404","UnitPrice":0.27},{"Amount":22.8,"Description":"CAR HANGING","ProductCode":"6815","UnitPrice":0.95},{"Amount":170.0,"Description":"KADA IMITATION","ProductCode":"1404","UnitPrice":1.7},{"Amount":166.8,"Description":"MERU SRI YANTRA","ProductCode":"83061000","UnitPrice":55.6},{"Amount":222.48,"Description":"MERU SRI YANTRA","ProductCode":"83061000","UnitPrice":18.54},{"Amount":225.12,"Description":"MERU SRI YANTRA","ProductCode":"83061000","UnitPrice":9.38},{"Amount":112.8,"Description":"MERU SRI YANTRA","ProductCode":"83061000","UnitPrice":4.7},{"Amount":66.0,"Description":"LOCKETS","ProductCode":"7117","UnitPrice":0.66},{"Amount":180.0,"Description":"POOJA CHOWKI","ProductCode":"74199930","UnitPrice":30.0},{"Amount":135.0,"Description

In [ ]:
# Our JSON data
json_data = item_part_of_invoice

# Parse JSON
data = json.loads(json_data)

# Function to find numbers with min 4 and max 10 characters in a string
def find_numbers(text):
    numbers = re.findall(r'\b\d{4,10}\b', str(text))  # \b ensures exact number of digits
    return numbers

# Iterate over each item
for item in data:
    # Find numbers in each field except "Amount"
    numbers = []
    for key, value in item.items():
        if key != "Amount":
            numbers.extend(find_numbers(value))

    # Convert the found numbers to a single string
    hsn_code = ' '.join(numbers)

    # Add HSN code to the item if numbers are found
    if hsn_code:
        item["HSN Code"] = hsn_code

# Convert back to JSON
result_json = json.dumps(data, indent=4)

print(result_json)

In [ ]:
print(r)

[RecognizedForm(form_type=prebuilt:invoice, fields={'CustomerAddress': FormField(value_type=string, label_data=None, value_data=FieldData(page_number=2, text=HARYANA 123501, bounding_box=[Point(x=1.1053, y=1.4963), Point(x=2.684, y=1.4963), Point(x=2.684, y=1.6023), Point(x=1.1053, y=1.6023)], field_elements=None), name=CustomerAddress, value='HARYANA 123501', confidence=0.946), 'CustomerAddressRecipient': FormField(value_type=string, label_data=None, value_data=FieldData(page_number=2, text=DENSO TEN MINDA INDIA PRIVATE LIMITED PLOT NO.383 INDUSTRIAL GRWOTH CENTRE, PHASE-2, SECTOR-3, BAWAL DISTT.REWARI, bounding_box=[Point(x=1.1057, y=0.8533), Point(x=4.7942, y=0.8533), Point(x=4.7942, y=1.4532), Point(x=1.1057, y=1.4532)], field_elements=None), name=CustomerAddressRecipient, value='DENSO TEN MINDA INDIA PRIVATE LIMITED PLOT NO.383 INDUSTRIAL GRWOTH CENTRE, PHASE-2, SECTOR-3, BAWAL DISTT.REWARI', confidence=0.868), 'CustomerId': FormField(value_type=string, label_data=None, value_data